I redid the raw data and need to re-cater the function to that.
Need to also add outputs for gender and pell

In [1]:
import pandas as pd
import numpy as np

Notes and explanations:
This function outputs 4 dataframes in respective order:

    - df1: the overall totals for all non-high school student transfers by FY/term

    - df2: the race, transfer count/rate, and group size by FY/term

    - df3: the firstgen status, transfer count/rate, and group size by FY/term

    - df4: each ind student's tech_id, transfer status, graduation status, cohort year/term, admission status, first gen status, and race


This function uses data from the below tables:

    - ISRS.ST_COU: the last semester a student attended RCC

    - ISRS.ST_TERM_MGMT: the admission status of the a student during their final RCC semester

    - APPSODS.V_STUDENT_PERSISTENCE_DTL2: each student's final record of retention, transfer, and graduation.

    - APPSODS.V_IPEDS_ETHNIC_RACE: what a students ethnicity/race is
    
    - APPSODS.V_FIRST_GENERATION: whether or not a student is first generation in their family to go to college

In [2]:
##retrieve data
persDTL2_10 = pd.read_csv('persDTL2_10.csv') #df1   now includes first gen, pell, and gender
courseRecords_10 = pd.read_csv('courseRecords_10.csv')
transferRecords_10 = pd.read_csv('transferRecords_10.csv')
race_10 = pd.read_csv('race_10.csv') #df4
admstat_10 = pd.read_csv('admstat_10.csv') #df3
last = pd.read_csv('lastsemester_10yrs.csv') #df2

In [3]:
persDTL2_10

,TECH_ID,COHORT_ENTERING_YEAR_TERM_CODE,FY,Semester,FIRST_GENERATION_STATUS_MN,FIRST_GENERATION_STATUS_FED,PELL_ELIGIBLE,GENDER,FIRST_FALL_RETAINED,FIRST_FALL_TRANSFERRED,...,EIGTH_FALL_RETAINED,EIGTH_FALL_TRANSFERRED,EIGTH_FALL_GRADUATED,EIGTH_FALL_TRANS_GRADUATED,EIGTH_FALL_SUCCESS,EIGTH_SPRING_RETAINED,EIGTH_SPRING_TRANSFERRED,EIGTH_SPRING_GRADUATED,EIGTH_SPRING_TRANS_GRADUATED,EIGTH_SPRING_SUCCESS
0,1185,20203,2020,3,No,Yes,Yes,Male,1,0,...,0,0,0,0,0,0,0,0,0,0
1,9722,20205,2020,5,No,Yes,Yes,Female,1,0,...,0,0,0,0,0,0,0,0,0,0
2,9929,20143,2014,3,Yes,Yes,Yes,Female,1,0,...,0,0,1,0,1,0,0,1,0,1
3,11387,20133,2013,3,Unknown,Unknown,Unknown,Male,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12943,20143,2014,3,No,Yes,No,Female,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,16532959,20243,2024,3,No,Yes,No,Female,0,0,...,0,0,0,0,0,0,0,0,0,0
9800,16534331,20243,2024,3,Unknown,Unknown,Yes,Female,0,0,...,0,0,0,0,0,0,0,0,0,0
9801,16539311,20243,2024,3,Unknown,Unknown,Unknown,Unknown,0,0,...,0,0,0,0,0,0,0,0,0,0
9802,16539765,20243,2024,3,No,No,Unknown,Male,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
persDTL = persDTL2_10.loc[persDTL2_10['FY']!=2024,:]
persDTL_fall = persDTL.loc[persDTL['Semester']==3,:]
persDTL_spring = persDTL.loc[persDTL['Semester']==5,:]

As it stands, this function currently works, outputs 4 dfs: overall by FY/term, race by FY/term, first gen by FY/term, student info

In [12]:
def fallTransferTally(df1,df2,df3,df4):
       trdf = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','COHORT_SIZE','TRAN_wi3','TRAN_at','TR'])
       trdf_race = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','RACE','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_FG = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','FIRST_GEN_MN','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_pell = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','PELL_ELG','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_gender = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','GENDER','COHORT_SIZE','TRAN_wi3','TR'])
       persDTL_2 = df1.copy()
       #create and fill new columns
       persDTL_2['transferred_wi3_fall'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED']
       persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
       persDTL_2['grad_at'] = persDTL_2['FIRST_FALL_GRADUATED'] + persDTL_2['FIRST_SPRING_GRADUATED'] + persDTL_2['SECOND_FALL_GRADUATED'] + persDTL_2['SECOND_SPRING_GRADUATED'] + persDTL_2['THIRD_FALL_GRADUATED'] + persDTL_2['THIRD_SPRING_GRADUATED'] + persDTL_2['FOURTH_FALL_GRADUATED'] + persDTL_2['FOURTH_SPRING_GRADUATED'] + persDTL_2['FIFTH_FALL_GRADUATED'] + persDTL_2['FIFTH_SPRING_GRADUATED'] + persDTL_2['SIXTH_FALL_GRADUATED'] + persDTL_2['SIXTH_SPRING_GRADUATED'] + persDTL_2['SEVENTH_FALL_GRADUATED'] + persDTL_2['SEVENTH_SPRING_GRADUATED'] + persDTL_2['EIGTH_FALL_GRADUATED'] + persDTL_2['EIGTH_SPRING_GRADUATED']
       persDTL_2['retained'] = persDTL_2['FIRST_FALL_RETAINED'] + persDTL_2['FIRST_SPRING_RETAINED'] + persDTL_2['SECOND_FALL_RETAINED'] + persDTL_2['SECOND_SPRING_RETAINED'] + persDTL_2['THIRD_FALL_RETAINED'] + persDTL_2['THIRD_SPRING_RETAINED'] + persDTL_2['FOURTH_FALL_RETAINED'] + persDTL_2['FOURTH_SPRING_RETAINED'] + persDTL_2['FIFTH_FALL_RETAINED'] + persDTL_2['FIFTH_SPRING_RETAINED'] + persDTL_2['SIXTH_FALL_RETAINED'] + persDTL_2['SIXTH_SPRING_RETAINED'] + persDTL_2['SEVENTH_FALL_RETAINED'] + persDTL_2['SEVENTH_SPRING_RETAINED'] + persDTL_2['EIGTH_FALL_RETAINED'] + persDTL_2['EIGTH_SPRING_RETAINED']
       #get rid of the lengthy semester columns
       persDTL_3 = persDTL_2[['TECH_ID','FIRST_GENERATION_STATUS_MN','FIRST_GENERATION_STATUS_FED','PELL_ELIGIBLE','GENDER','FY','Semester','transferred_wi3_fall','transferred_at','grad_at','retained']]
       #merge df2 through df5
       last_admstat = pd.merge(df2, df3, how='left',on=['TECH_ID','YRTR'])
       last_admstat_race = pd.merge(last_admstat, df4,how='left',on='TECH_ID')
       #merge with the transfer df
       demo_transfers = pd.merge(persDTL_3, last_admstat_race, how='left',on='TECH_ID')
       fys = demo_transfers['FY'].unique()
       for fy in fys:
              df_main = demo_transfers.loc[(demo_transfers['FY']==fy),:]
              ##df_out1 variables
              cohort_size_main, transferred_wi3_main, transferred_at_main = 0,0,0
              ##df_out2 variables
              two_plus,native_a, pacific_i, hispanic, asian, white, black, nra, unknown = 0,0,0,0,0,0,0,0,0
              two_plus_t, native_a_t, pacific_i_t, hispanic_t, asian_t, white_t, black_t, nra_t, unknown_t = 0,0,0,0,0,0,0,0,0
              TR_white,TR_asian,TR_black,TR_hispanic,TR_two_plus,TR_native_a,TR_pacific_i,TR_nra,TR_unknown = 0,0,0,0,0,0,0,0,0
              ##df_out3 variables
              fg_cs, fg_t = 0,0
              ##df_out4 variables
              pell_elg, pell_elg_t = 0,0
              ##df_out5 variables
              male, female, male_t, female_t = 0,0,0,0
              df_main.reset_index()
              for i in df_main.index:
                     cohort_size_main = cohort_size_main + 1
                     at = df_main.loc[i,'transferred_at']
                     wi3 = df_main.loc[i,'transferred_wi3_fall']
                     fg = df_main.loc[i,'FIRST_GENERATION_STATUS_MN']
                     race = df_main.loc[i,'IPEDS']
                     pell = df_main.loc[i,'PELL_ELIGIBLE']
                     gender = df_main.loc[i,'GENDER']
                     if race == 'WHITE':
                            white = white + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   white_t = white_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'HISP':
                            hispanic = hispanic + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   hispanic_t = hispanic_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'BLACK':
                            black = black + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   black_t = black_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'ASIAN':
                            asian = asian + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   asian_t = asian_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'TWO_PLUS':
                            two_plus = two_plus + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   two_plus_t = two_plus_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'AMER_IND_AK':
                            native_a = native_a + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   native_a_t = native_a_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NAT_HAW_PI':
                            pacific_i = pacific_i + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   pacific_i_t = pacific_i_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NRA':
                            nra = nra + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   nra_t = nra_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'UNKNOWN':
                            unknown = unknown + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   unknown_t = unknown_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1       
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     else:
                            continue
              ##Race transfer rate; this doesn't seem necessary, but some of these groupings are 0 size, which will break that function should a 0 end up in the denominator 
              if white > 0:
                     TR_white = white_t/white*100   
              if asian > 0:
                     TR_asian = asian_t/asian*100   
              if black > 0:
                     TR_black = black_t/black*100   
              if hispanic > 0:
                     TR_hispanic = hispanic_t/hispanic*100   
              if two_plus > 0:
                     TR_two_plus = two_plus_t/two_plus*100 
              if native_a > 0:  
                     TR_native_a = native_a_t/native_a*100   
              if pacific_i > 0:
                     TR_pacific_i = pacific_i_t/pacific_i*100   
              if nra > 0:
                     TR_nra = nra_t/nra*100   
              if unknown > 0:
                     TR_unknown = unknown_t/unknown*100  
              #df: gender
              TR_fem = 0
              if female > 0:
                     TR_fem = female_t/female*100
              TR_male = 0
              if male > 0:
                     TR_male = male_t/male*100
              sdf_gender = pd.DataFrame(
                     {'COHORT_FY':[fy,fy],
                     'COHORT_SEMESTER':['FALL','FALL'],
                     'GENDER':['Female','Male'],
                     'COHORT_SIZE':[female, male],
                     'TRAN_wi3':[female_t, male_t],
                     'TR':[TR_fem, TR_male]}, index=[1,2])
              #df: pell
              TR_pell = 0
              if pell_elg > 0:
                     TR_pell = pell_elg_t/pell_elg*100
              sdf_pell = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['FALL'],
                     'PELL_ELG':['Yes'],
                     'COHORT_SIZE':[pell_elg],
                     'TRAN_wi3':[pell_elg_t],
                     'TR':[TR_pell]}, index=[1])
              #df: race
              sdf_race = pd.DataFrame(
                     {'COHORT_FY':[fy,fy,fy,fy,fy,fy,fy,fy,fy],
                     'COHORT_SEMESTER':['FALL','FALL','FALL','FALL','FALL','FALL','FALL','FALL','FALL'],
                     'RACE':['WHITE','ASIAN','HISP','BLACK','TWO_PLUS','AMER_IND_AK','NAT_HAW_PI','NRA','UNKNOWN'],
                     'COHORT_SIZE':[white, asian, hispanic, black, two_plus, native_a, pacific_i, nra, unknown],
                     'TRAN_wi3':[white_t, asian_t, hispanic_t, black_t, two_plus_t, native_a_t, pacific_i_t, nra_t, unknown_t],
                     'TR':[TR_white, TR_asian, TR_hispanic, TR_black, TR_two_plus, TR_native_a, TR_pacific_i, TR_nra, TR_unknown]}, 
                     index = [1,2,3,4,5,6,7,8,9])
              ##df: first generation
              TR_fg = 0
              if fg_cs > 0:
                     TR_fg = fg_t/fg_cs*100 
              sdf_fg = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['FALL'],
                     'FIRST_GEN_MN':['Yes'],
                     'COHORT_SIZE':[fg_cs],
                     'TRAN_wi3':[fg_t],
                     'TR':[TR_fg]}, index=[1])
              ##df: overall
              TR = transferred_wi3_main/cohort_size_main*100
              sdf = pd.DataFrame(
                     {'COHORT_FY':fy,
                     'COHORT_SEMESTER':'FALL',
                     'COHORT_SIZE':cohort_size_main,
                     'TRAN_wi3':transferred_wi3_main,
                     'TRAN_at':transferred_at_main,
                     'TR':TR}, index=[1])
              ##apppend all new dfs onto the main ones
              trdf_gender = pd.concat([trdf_gender, sdf_gender])
              trdf_pell = pd.concat([trdf_pell, sdf_pell])
              trdf_race = pd.concat([trdf_race,sdf_race])
              trdf_FG = pd.concat([trdf_FG,sdf_fg])
              trdf = pd.concat([trdf,sdf])
       return trdf, trdf_gender, trdf_pell, trdf_FG, trdf_race, demo_transfers
    

In [25]:
def springTransferTally(df1,df2,df3,df4):
       trdf = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','COHORT_SIZE','TRAN_wi3','TRAN_at','TR'])
       trdf_race = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','RACE','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_FG = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','FIRST_GEN_MN','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_pell = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','PELL_ELG','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_gender = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','GENDER','COHORT_SIZE','TRAN_wi3','TR'])
       persDTL_2 = df1.copy()
       #create and fill new columns
       persDTL_2['transferred_wi3_spring'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED']
       persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
       persDTL_2['grad_at'] = persDTL_2['FIRST_FALL_GRADUATED'] + persDTL_2['FIRST_SPRING_GRADUATED'] + persDTL_2['SECOND_FALL_GRADUATED'] + persDTL_2['SECOND_SPRING_GRADUATED'] + persDTL_2['THIRD_FALL_GRADUATED'] + persDTL_2['THIRD_SPRING_GRADUATED'] + persDTL_2['FOURTH_FALL_GRADUATED'] + persDTL_2['FOURTH_SPRING_GRADUATED'] + persDTL_2['FIFTH_FALL_GRADUATED'] + persDTL_2['FIFTH_SPRING_GRADUATED'] + persDTL_2['SIXTH_FALL_GRADUATED'] + persDTL_2['SIXTH_SPRING_GRADUATED'] + persDTL_2['SEVENTH_FALL_GRADUATED'] + persDTL_2['SEVENTH_SPRING_GRADUATED'] + persDTL_2['EIGTH_FALL_GRADUATED'] + persDTL_2['EIGTH_SPRING_GRADUATED']
       persDTL_2['retained'] = persDTL_2['FIRST_FALL_RETAINED'] + persDTL_2['FIRST_SPRING_RETAINED'] + persDTL_2['SECOND_FALL_RETAINED'] + persDTL_2['SECOND_SPRING_RETAINED'] + persDTL_2['THIRD_FALL_RETAINED'] + persDTL_2['THIRD_SPRING_RETAINED'] + persDTL_2['FOURTH_FALL_RETAINED'] + persDTL_2['FOURTH_SPRING_RETAINED'] + persDTL_2['FIFTH_FALL_RETAINED'] + persDTL_2['FIFTH_SPRING_RETAINED'] + persDTL_2['SIXTH_FALL_RETAINED'] + persDTL_2['SIXTH_SPRING_RETAINED'] + persDTL_2['SEVENTH_FALL_RETAINED'] + persDTL_2['SEVENTH_SPRING_RETAINED'] + persDTL_2['EIGTH_FALL_RETAINED'] + persDTL_2['EIGTH_SPRING_RETAINED']
       #get rid of the lengthy semester columns
       persDTL_3 = persDTL_2[['TECH_ID','FIRST_GENERATION_STATUS_MN','FIRST_GENERATION_STATUS_FED','PELL_ELIGIBLE','GENDER','FY','Semester','transferred_wi3_spring','transferred_at','grad_at','retained']]
       #merge df2 through df5
       last_admstat = pd.merge(df2, df3, how='left',on=['TECH_ID','YRTR'])
       last_admstat_race = pd.merge(last_admstat, df4,how='left',on='TECH_ID')
       #merge with the transfer df
       demo_transfers = pd.merge(persDTL_3, last_admstat_race, how='left',on='TECH_ID')
       fys = demo_transfers['FY'].unique()
       for fy in fys:
              df_main = demo_transfers.loc[(demo_transfers['FY']==fy),:]
              ##df_out1 variables
              cohort_size_main, transferred_wi3_main, transferred_at_main = 0,0,0
              ##df_out2 variables
              two_plus,native_a, pacific_i, hispanic, asian, white, black, nra, unknown = 0,0,0,0,0,0,0,0,0
              two_plus_t, native_a_t, pacific_i_t, hispanic_t, asian_t, white_t, black_t, nra_t, unknown_t = 0,0,0,0,0,0,0,0,0
              TR_white,TR_asian,TR_black,TR_hispanic,TR_two_plus,TR_native_a,TR_pacific_i,TR_nra,TR_unknown = 0,0,0,0,0,0,0,0,0
              ##df_out3 variables
              fg_cs, fg_t = 0,0
              ##df_out4 variables
              pell_elg, pell_elg_t = 0,0
              ##df_out5 variables
              male, female, male_t, female_t = 0,0,0,0
              df_main.reset_index()
              for i in df_main.index:
                     cohort_size_main = cohort_size_main + 1
                     at = df_main.loc[i,'transferred_at']
                     wi3 = df_main.loc[i,'transferred_wi3_spring']
                     fg = df_main.loc[i,'FIRST_GENERATION_STATUS_MN']
                     race = df_main.loc[i,'IPEDS']
                     pell = df_main.loc[i,'PELL_ELIGIBLE']
                     gender = df_main.loc[i,'GENDER']
                     if race == 'WHITE':
                            white = white + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   white_t = white_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'HISP':
                            hispanic = hispanic + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   hispanic_t = hispanic_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'BLACK':
                            black = black + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   black_t = black_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'ASIAN':
                            asian = asian + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   asian_t = asian_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'TWO_PLUS':
                            two_plus = two_plus + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   two_plus_t = two_plus_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'AMER_IND_AK':
                            native_a = native_a + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   native_a_t = native_a_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NAT_HAW_PI':
                            pacific_i = pacific_i + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   pacific_i_t = pacific_i_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NRA':
                            nra = nra + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   nra_t = nra_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'UNKNOWN':
                            unknown = unknown + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   unknown_t = unknown_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1       
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     else:
                            continue
              ##Race transfer rate; this doesn't seem necessary, but some of these groupings are 0 size, which will break that function should a 0 end up in the denominator 
              if white > 0:
                     TR_white = white_t/white*100   
              if asian > 0:
                     TR_asian = asian_t/asian*100   
              if black > 0:
                     TR_black = black_t/black*100   
              if hispanic > 0:
                     TR_hispanic = hispanic_t/hispanic*100   
              if two_plus > 0:
                     TR_two_plus = two_plus_t/two_plus*100 
              if native_a > 0:  
                     TR_native_a = native_a_t/native_a*100   
              if pacific_i > 0:
                     TR_pacific_i = pacific_i_t/pacific_i*100   
              if nra > 0:
                     TR_nra = nra_t/nra*100   
              if unknown > 0:
                     TR_unknown = unknown_t/unknown*100  
              #df: gender
              TR_fem = 0
              if female > 0:
                     TR_fem = female_t/female*100
              TR_male = 0
              if male > 0:
                     TR_male = male_t/male*100
              sdf_gender = pd.DataFrame(
                     {'COHORT_FY':[fy,fy],
                     'COHORT_SEMESTER':['SPRING','SPRING'],
                     'GENDER':['Female','Male'],
                     'COHORT_SIZE':[female, male],
                     'TRAN_wi3':[female_t, male_t],
                     'TR':[TR_fem, TR_male]}, index=[1,2])
              #df: pell
              TR_pell = 0
              if pell_elg > 0:
                     TR_pell = pell_elg_t/pell_elg*100
              sdf_pell = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['SPRING'],
                     'PELL_ELG':['Yes'],
                     'COHORT_SIZE':[pell_elg],
                     'TRAN_wi3':[pell_elg_t],
                     'TR':[TR_pell]}, index=[1])
              #df: race
              sdf_race = pd.DataFrame(
                     {'COHORT_FY':[fy,fy,fy,fy,fy,fy,fy,fy,fy],
                     'COHORT_SEMESTER':['SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING'],
                     'RACE':['WHITE','ASIAN','HISP','BLACK','TWO_PLUS','AMER_IND_AK','NAT_HAW_PI','NRA','UNKNOWN'],
                     'COHORT_SIZE':[white, asian, hispanic, black, two_plus, native_a, pacific_i, nra, unknown],
                     'TRAN_wi3':[white_t, asian_t, hispanic_t, black_t, two_plus_t, native_a_t, pacific_i_t, nra_t, unknown_t],
                     'TR':[TR_white, TR_asian, TR_hispanic, TR_black, TR_two_plus, TR_native_a, TR_pacific_i, TR_nra, TR_unknown]}, 
                     index = [1,2,3,4,5,6,7,8,9])
              ##df: first generation
              TR_fg = 0
              if fg_cs > 0:
                     TR_fg = fg_t/fg_cs*100 
              sdf_fg = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['SPRING'],
                     'FIRST_GEN_MN':['Yes'],
                     'COHORT_SIZE':[fg_cs],
                     'TRAN_wi3':[fg_t],
                     'TR':[TR_fg]}, index=[1])
              ##df: overall
              TR = transferred_wi3_main/cohort_size_main*100
              sdf = pd.DataFrame(
                     {'COHORT_FY':fy,
                     'COHORT_SEMESTER':'SPRING',
                     'COHORT_SIZE':cohort_size_main,
                     'TRAN_wi3':transferred_wi3_main,
                     'TRAN_at':transferred_at_main,
                     'TR':TR}, index=[1])
              ##apppend all new dfs onto the main ones
              trdf_gender = pd.concat([trdf_gender, sdf_gender])
              trdf_pell = pd.concat([trdf_pell, sdf_pell])
              trdf_race = pd.concat([trdf_race,sdf_race])
              trdf_FG = pd.concat([trdf_FG,sdf_fg])
              trdf = pd.concat([trdf,sdf])
       return trdf, trdf_gender, trdf_pell, trdf_FG, trdf_race, demo_transfers
    

In [34]:
d1,d2,d3,d4,d5,d6 = fallTransferTally(persDTL_fall, last, admstat_10, race_10)
d7,d8,d9,d10,d11,d12 = springTransferTally(persDTL_spring, last, admstat_10, race_10)


In [35]:
d13 = pd.concat([d1,d7])
d14 = pd.concat([d2,d8])
d15 = pd.concat([d3,d9])
d16 = pd.concat([d4,d10])
d17 = pd.concat([d5,d11])
d18 = pd.concat([d6,d12])

In [40]:
d13.to_csv('d13.csv', index=False)
d14.to_csv('d14.csv', index=False)
d15.to_csv('d15.csv', index=False)
d16.to_csv('d16.csv', index=False)
d17.to_csv('d17.csv', index=False)
d18.to_csv('d18.csv', index=False)

In [39]:
#d13.sort_values(by=['FY','Semester','transferred_wi3','TECH_ID'])
d13.sort_values(by=['COHORT_FY','COHORT_SEMESTER'])
#d11.sort_values(by=['COHORT_FY','COHORT_SEMESTER'])


,COHORT_FY,COHORT_SEMESTER,COHORT_SIZE,TRAN_wi3,TRAN_at,TR
1,2013,FALL,748,109,144,14.572193
1,2013,SPRING,233,60,71,25.751073
1,2014,FALL,669,121,159,18.086697
1,2014,SPRING,243,45,57,18.518519
1,2015,FALL,616,108,142,17.532468
1,2015,SPRING,161,38,45,23.602484
1,2016,FALL,566,97,132,17.137809
1,2016,SPRING,161,23,32,14.285714
1,2017,FALL,620,79,114,12.741935
1,2017,SPRING,142,25,28,17.605634
